# Fortran magic & OpenMP

Load the Fortran magic extension.

In [1]:
%load_ext fortranmagic

/home/gjb/miniconda3/envs/fortran/lib/python3.6/site-packages/fortranmagic.py:147: UserWarning: get_ipython_cache_dir has moved to the IPython.paths module since IPython 4.0.
  self._lib_dir = os.path.join(get_ipython_cache_dir(), 'fortran')


Check the number of threads for OpenMP, if it is 1, things will not be very interesting.  Set before starting the notebook using:
```
export OMP_NUM_THREADS=3
```

In [2]:
import os
os.environ['OMP_NUM_THREADS']

'3'

A very simple subroutine that illustrates that OpenMP works.  It returns the number of threads, and the product $\prod_{t_{\rm id}=0}^{t_{\rm num} - 1} (t_{\rm id} + 1)$ for $t_{\rm id}$ the thread ID and $t_{\rm num}$ the number of threads.

In [18]:
%%fortran --f90flags '-fopenmp'
subroutine test_omp(num_threads, thread_num_prod)
    use omp_lib
    implicit none
    integer, intent(out) :: num_threads, thread_num_prod
    integer :: thread_num
    thread_num_prod = 1
    !$omp parallel private(thread_num), shared(num_threads), reduction(*:thread_num_prod)
        thread_num = omp_get_thread_num()
        if (thread_num == 0) &
            num_threads = omp_get_num_threads()
        thread_num_prod = thread_num_prod*(1 + thread_num)
    !$omp end parallel
end subroutine test_omp

For three threads, the results should be 3 and 6 respectively.

In [19]:
test_omp()

(3, 6)